<a href="https://colab.research.google.com/github/Nishant5050/FletasTest/blob/main/sslexpiry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install pyOpenSSL
import ssl
from datetime import datetime
import pytz
import OpenSSL
import socket
import getpass
from datetime import timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

print('Program to check SSL certificate validity \n')
ipfile=open('/content/nish.txt')
cur_date = datetime.utcnow()
mailbody=''
expcount=0

##getting details
expday=input('Please provide threshold expiry date:')
from_mail=input('Your mail id :')
passwd=getpass.getpass('password :' )
to_mail=input('Target mail id : ')

for ip in ipfile:
    try:
        host = ip.strip().split(':')[0]
        port = ip.strip().split(':')[1]
        print('\nChecking certifcate for server', host)
        ctx = OpenSSL.SSL.Context(ssl.PROTOCOL_TLSv1)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((host, int(port)))
        cnx = OpenSSL.SSL.Connection(ctx, s)
        cnx.set_connect_state()
        cnx.do_handshake()
        cert = cnx.get_peer_certificate()
        s.close()
        server_name = cert.get_subject().commonName
        print(server_name)
        ##checking expiry date
        edate = cert.get_notAfter()
        edate = edate.decode()
        ##converting in to system time format
        exp_date = datetime.strptime(edate, '%Y%m%d%H%M%SZ')
        days_to_expire = int((exp_date - cur_date).days)
        print('day to expire', days_to_expire)
        ##preparing mail body
        if days_to_expire < int(expday):
            expcount = expcount + 1
            mailbody = mailbody + '\n Server name =' + server_name + ', Days to expire:' + str(days_to_expire)
    except:
        print ('error on connection to Server,',host)
        print (mailbody)

# sending mail if any certificate going to expire within threshold days
    if expcount >= 1:
        try:
            print('\nCertifcate alert for' +str(expcount)+' Servers, Sending mails')
            body ='Following certificate going to expire, please take action \n'+mailbody
            s = smtplib.SMTP(host='smtp - mail.outlook.com', port = 587)  # change here if you want to use other smtp server
            s.starttls()
            s.login(from_mail, passwd)
            msg = MIMEMultipart()  # create a message
            msg['From']=from_mail
            msg['To']=to_mail
            msg['Subject']='Certificate Expire alert'
            msg.attach(MIMEText(str(body),'plain'))
            s.send_message(msg)
            print('Mail sent')
            s.close()
        except:
            print('Sending mail failed')
    else:
        print('All certificate are below the threshold date')
        print('\nCertification check completed')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Program to check SSL certificate validity 

Please provide threshold expiry date:200
Your mail id :nishantshete5050@gmail.com
password :··········
Target mail id : shetenishant5050@gmail.com

Checking certifcate for server www.google.com
invalid2.invalid
day to expire 2661
All certificate are below the threshold date

Certification check completed

Checking certifcate for server www.yahoo.com
yahoo.com
day to expire 108

Certifcate alert for1 Servers, Sending mails
Sending mail failed

Checking certifcate for server www.gmail.com
invalid2.invalid
day to expire 2661

Certifcate alert for1 Servers, Sending mails
Sending mail failed
